In [1]:
#PopFM
reorderG<-read.table("GS2018/reorderG.txt")
x1<-as.matrix(reorderG)-1
#PopLFM1
reorderG<-read.table("GS2019/reorderG.txt")
x2<-as.matrix(reorderG)-1
#PopLFM2
reorderG<-read.table("GS2020/reorderG.txt")
x3<-as.matrix(reorderG)-1

# Dim of marker matrix in each population
dim(x1)
dim(x2)
dim(x3)
n1<-dim(x1)[1]
n2<-dim(x2)[1]
n3<-dim(x3)[1]

# Shared loci across 3 populations
m<-intersect(colnames(x1),colnames(x2))
m2<-intersect(m,colnames(x3))
length(m2)

x1_intersect<-x1[,(colnames(x1)%in%m2)]
x2_intersect<-x2[,(colnames(x2)%in%m2)]
x3_intersect<-x3[,(colnames(x3)%in%m2)]
x_all<-rbind(x1_intersect,x2_intersect,x3_intersect)
str(x_all)

[1]  1100 12548

[1]   993 19227

[1]  1017 21272

[1] 4646

 num [1:3110, 1:4646] 1 0 0 0 -1 -1 0 -1 1 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:3110] "1" "2" "3" "4" ...
  ..$ : chr [1:4646] "chr9.7831_G" "chr9.7851_T" "chr9.7866_T" "chr9.7956_G" ...


In [48]:
# Fst (Hudson et al., 1992) 
# Hudson, R.R., Slatkin, M., Maddison, W.P., 1992. Estimation of levels of gene flow from DNA-sequence data. Genetics 132(2), 583-589. https://doi.org/10.1093/genetics/132.2.583.
# modified from: https://bios1140.github.io/understanding-fst-the-fixation-index.html

# a simple function to calculate allele frequencies in two populations
calc_af <- function(counts){
  # get the number of samples
  n <- length(counts)
  # calculate frequency of 1st allele - p
  p <- ((sum(counts==-1)*2) + sum(counts==-0))/(2*n)
  return(p)
}

# a function to calculate fst
calc_fst <- function(p_1, p_2){
  
  # calculate q1 and q2
  q_1 <- 1 - p_1
  q_2 <- 1 - p_2
  
  # calculate total allele frequency
  p_t <- (p_1 + p_2)/2
  q_t <- 1 - p_t
  
  # calculate expected heterozygosity
  # first calculate expected heterozygosity for the two populations
  # pop1
  hs_1 <- 2*p_1*q_1
  # pop2
  hs_2 <- 2*p_2*q_2
  # then take the mean of this
  hs <- (hs_1 + hs_2)/2
  
  # next calculate expected heterozygosity for the metapopulations
  ht <- 2*p_t*q_t
  
  # calculate fst
  fst <- (ht - hs)/ht
  
  # return output
  return(fst)
}

# Test
# calc_fst(calc_af(x1_intersect[,1]),calc_af(x2_intersect[,1])) 

In [53]:
p1 <- apply(x1_intersect, 2, calc_af)
p2 <- apply(x2_intersect, 2, calc_af)
p3 <- apply(x3_intersect, 2, calc_af)

In [57]:
Fst12<-c()
for (i in 1:length(p1)){
    Fst12<-c(Fst12,calc_fst(p1[i],p2[i]))
}
Fst13<-c()
for (i in 1:length(p1)){
    Fst13<-c(Fst13,calc_fst(p1[i],p3[i]))
}
Fst23<-c()
for (i in 1:length(p1)){
    Fst23<-c(Fst23,calc_fst(p2[i],p3[i]))
}

In [58]:
#PopFM-PopLFM1
mean(Fst12)
#PopFM-PopLFM2
mean(Fst13)
#PopLFM1-PopLFM2
mean(Fst23)

[1] 0.006546875

[1] 0.009151627

[1] 0.007017098